# Nya features
### 1. Kolla vad antal hästar per lopp kan ge
### 2. Kolla vad Bayes target kodning för bana, kusk och häst kan ge

In [ ]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# Läs in all_data.csv
# select numeric columns
df = pd.read_csv('../all_data.csv')
# df.set_index('datum', inplace=True)
df.columns

In [ ]:
# count number per avdeling
df_count = df.groupby(['datum','avd'])['avd'].count()
df['avd_count'] = None
df['avd_count'] = df.groupby(['datum', 'avd'])['avd'].transform('count')

In [ ]:
df['avd_count'].unique()
my_remove_list = ['streck']
sorted(df['avd_count'].unique())

# Kolla med catboost

### Några hjälp-funktioner

In [ ]:
def split_data(df_,train_from_proc=0,test_proc=0.25):
    df=df_.copy()
    alla_datum = df.datum.unique()
    train_from_datum = alla_datum[ int(len(alla_datum)*train_from_proc)]
    print("train from",train_from_datum)
    X_test=None
    y_test=None
    test_from_datum=alla_datum[-1]
    if test_proc:
        selected_data = alla_datum[ alla_datum >= train_from_datum ]
        test_from_datum = selected_data[ int(len(selected_data)*(1-test_proc)) ]
        print("test from",test_from_datum)
        X_test  = df[df.datum >= test_from_datum]
        y_test  = (X_test.plac==1)*1
        X_test  = X_test.drop('plac',axis=1)
        
    X_train = df[(df.datum >= train_from_datum) & (df.datum < test_from_datum) ]
    y_train = (X_train.plac==1)*1
    
    return X_train.drop('plac', axis=1), X_test, y_train, y_test
### Features som inte används vid träning

def remove_features(df_, remove_mer=[]):
    df = df_.drop(['avd', 'startnr', 'vodds', 'podds', 'bins',
                  'h1_dat', 'h2_dat', 'h3_dat', 'h4_dat', 'h5_dat'], axis=1)
    if remove_mer:
        df = df.drop(remove_mer, axis=1)
    return df


### set up for catboost classifier

In [ ]:
# set up catboost classifier model
from catboost import CatBoostClassifier, cv, Pool

X_train, X_test, y_train, y_test = split_data(
    df, train_from_proc=0.3, test_proc=0.25)
X_train = remove_features(X_train, remove_mer=my_remove_list )
X_test = remove_features(X_test, remove_mer=my_remove_list )

cat_features = list(X_train.select_dtypes('object').columns)
X_train[cat_features] = X_train[cat_features].fillna('missing')
X_test[cat_features] = X_test[cat_features].fillna('missing')
print(cat_features)
pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)
X_train.shape, X_test.shape


In [ ]:
X_train.info()

In [ ]:
# run catboost
cb = CatBoostClassifier(
    iterations=1000,
    # eval_metric='MAE',
    early_stopping_rounds=100,
    random_seed=42,
    verbose=100
)
cb.fit(pool, eval_set=test_pool)

Med streck:  
alla:  
bestTest = 0.2410183541  
bestIteration = 210  

alla 0:or borta:  
bestTest = 0.2412283588  
bestIteration = 67  

Utan streck:  
alla:  
bestTest = 0.2745510195  
bestIteration = 0 

alla 0:or borta:  
bestTest =0.272817442 0.2742631409   
bestIteration = 75  






In [ ]:
my_feature_imp =cb.get_feature_importance(prettified=True)
my_remove_list2 = my_feature_imp[my_feature_imp.Importances <= 0]['Feature Id'].to_list()
print(my_remove_list2)
my_remove_list=my_remove_list+my_remove_list2
# remove duplicates
my_remove_list = list(set(my_remove_list))


In [ ]:
cb.get_feature_importance(prettified=True)

# Använd Bayes target encoding (mitt eget namn) 
### för Bana, Häst och Kusk (fortsätt senare)

In [ ]:
# Bayes function
def compute_Bayes(bana, häst, df_):
    df = df_.copy()
    # p(A|B) = p(B|A) * p(A) / p(B)
    # p(A) häst total performance
    # p(B) Share of bana in df
    # p(B|A) Share of bana where häst is winner
    # p(A|B) probabiity that häst is winner in bana
    # Find häst total performance
    total_performance = df.loc[(df.häst == häst) & (df.plac == 1),['datum','avd','bana','häst','kusk']]
    print(total_performance)
compute_Bayes('AXEVALLA', 'NINJA ZON', df)

# df.loc[(df.häst == 'GOGO GIRL') & (df.plac == 1),'plac']
print(f'Antal ggr NUNCIO har vunnit på SOLVALLA  : {df.loc[(df.bana=="SOLVALLA") & (df.häst=="NUNCIO") & (df.plac==1),"häst"].count()}')
print(f"Antal ggr NUNCIO har sprungit på SOLVALLA: {df.loc[(df.bana == 'SOLVALLA') & (df.häst == 'NUNCIO'), 'häst'].count()}")
print(f"Antal ggr NUNCIO har vunnit totalt       : {df.loc[(df.häst == 'NUNCIO') & (df.plac==1),'häst'].count()}")
print(f"Antal ggr NUNCIO har sprungit totalt     : {df.loc[(df.häst == 'NUNCIO'),'häst'].count()}")
